[View in Colaboratory](https://colab.research.google.com/github/robmoore/diabetes-prediction/blob/master/Create_CCS_SQLite_Database.ipynb)

In [9]:
!apt install sqlite3 -q

Reading package lists...
Building dependency tree...
Reading state information...
sqlite3 is already the newest version (3.19.3-3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [0]:
csvfile = 'dxref_2015.csv'
!curl -s 'http://www.nber.org/ahrq/ccs/2017/20170421/%24dxref%202015.csv' | tail -n +2 | sed "s/'/\"/g" > {csvfile}

In [0]:
import pandas as pd
import sqlite3
import numpy as np
  
from contextlib import closing

# Data uploaded to Google Drive below. Assumes the existence of a folder named 'Diabetes-Prediction' in your Google Drive account.
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
!head {csvfile}

"ICD-9-CM CODE","CCS CATEGORY","CCS CATEGORY DESCRIPTION","ICD-9-CM CODE DESCRIPTION","OPTIONAL CCS CATEGORY","OPTIONAL CCS CATEGORY DESCRIPTION"
"     ","0    ","No DX","INVALID CODES IN USER DATA"," "," "
"01000","1    ","Tuberculosis","PRIM TB COMPLEX-UNSPEC"," "," "
"01001","1    ","Tuberculosis","PRIM TB COMPLEX-NO EXAM"," "," "
"01002","1    ","Tuberculosis","PRIM TB COMPLEX-EXM UNKN"," "," "
"01003","1    ","Tuberculosis","PRIM TB COMPLEX-MICRO DX"," "," "
"01004","1    ","Tuberculosis","PRIM TB COMPLEX-CULT DX"," "," "
"01005","1    ","Tuberculosis","PRIM TB COMPLEX-HISTO DX"," "," "
"01006","1    ","Tuberculosis","PRIM TB COMPLEX-OTH TEST"," "," "
"01010","1    ","Tuberculosis","PRIM TB PLEURISY-UNSPEC"," "," "


In [0]:
db_name = 'dxref_2015.db'
table_name = 'dxref'

df = pd.read_csv(csvfile)
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df = df.applymap(lambda x: x.strip() if type(x) is str else x)
df.replace('', np.NaN)  
      
with closing(sqlite3.connect('file:{}'.format(db_name), uri=True)) as conn:    
  df.to_sql(table_name, conn, if_exists='replace', index=False)

In [5]:
df.head()

,ICD_9_CM_CODE,CCS_CATEGORY,CCS_CATEGORY_DESCRIPTION,ICD_9_CM_CODE_DESCRIPTION,OPTIONAL_CCS_CATEGORY,OPTIONAL_CCS_CATEGORY_DESCRIPTION
0,,0,No DX,INVALID CODES IN USER DATA,,
1,01000,1,Tuberculosis,PRIM TB COMPLEX-UNSPEC,,
2,01001,1,Tuberculosis,PRIM TB COMPLEX-NO EXAM,,
3,01002,1,Tuberculosis,PRIM TB COMPLEX-EXM UNKN,,
4,01003,1,Tuberculosis,PRIM TB COMPLEX-MICRO DX,,


In [6]:
groups = df.groupby(['CCS_CATEGORY'])['ICD_9_CM_CODE'].apply(list)
groups.loc[49]

['24900',
 '25000',
 '25001',
 '7902',
 '79021',
 '79022',
 '79029',
 '7915',
 '7916',
 'V4585',
 'V5391',
 'V6546']

In [7]:
# See also: Screen DM -> V771, FHX -> V180, NEUROPATHY IN DIABETES -> 3572
with closing(sqlite3.connect('file:{}?mode=ro'.format(db_name), uri=True)) as conn:    
    dxref = pd.read_sql_query("""
    SELECT CCS_CATEGORY, ICD_9_CM_CODE
    FROM {}
    WHERE CCS_CATEGORY IN ('49', '50', '186')
    GROUP BY ICD_9_CM_CODE
    ORDER BY CCS_CATEGORY
""".format(table_name), conn)
dxref.head()

,CCS_CATEGORY,ICD_9_CM_CODE
0,49,24900
1,49,25000
2,49,25001
3,49,7902
4,49,79021


In [10]:
!rm {csvfile}
!cp {db_name} /content/gdrive/My\ Drive/Diabetes-Prediction/

rm: cannot remove 'dxref_2015.csv': No such file or directory
